# モデルの学習
本単元では、前回作成した**MLPモデルを学習させる過程**について説明する。  
具体的には、学習に使う`train()`関数とテストに使う`test()`関数の実装だ。  

## この単元の目標
- MLPモデルを学習できるようになろう。
- MLPモデルをテストできるようになろう。

## 0. 準備

In [ ]:
# 本章で使うモジュール
from torch import utils
from torchvision import datasets
import torchvision.transforms as transforms

import torch
from torch import nn, optim
from torch.nn import functional as F

### 【MNISTデータセット】
まずは、学習に使うデータセットを準備しよう。  
**MNISTデータセット**は、**画像データと正解ラベルがセットになったクラス分類のための教師ありデータセット**だ。  
画像には**0〜9の数字**が書かれ、正解ラベルはその画像が表している1桁の数字であるため、**10クラスの分類**である。  
1枚の画像の形状は、色が白黒で28\*28画素なので、その**形状は`(28, 28, 1)`**だ。  

pytorchには、データセットがライブラリとしていくつか用意（実際にはダウンロードする関数）されており、  
MNISTもその1つだ。  
以下のコードを実行すれば、データセットをダウンロードする事ができる。

In [ ]:
from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

## こうやってダウンロードして使うことができるよ
trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
train_loader = utils.data.DataLoader(trainset, batch_size=100, shuffle=True, num_workers=2)

testset = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())
test_loader = utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

### 【モデルの準備】
次に、学習を行うためのモデルを準備しよう。  
行うタスクは、**「28*28画素の白黒画像を10クラス分類する」**事なので、  
入力ノードの数は784（=28*28）個で、出力ノードの数は10個で良いだろう。

よって、以下の条件のMLPモデルをクラスとして定義する。  
- 「入力層、中間層、出力層」のノードの数が「784（=28×28）、512、10」
- 中間層の出力に活性化関数`relu()`を適用
- 損失関数は`MSELoss()`
- 最適化関数は`Adam()`

In [ ]:
class mlp_net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 10)

        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.parameters())

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

## 1. train()関数の実装

それでは、**`train()`関数の実装**を行っていこう。  
大枠は、「実用画像データ処理コース」で実装したものと同じだが、下記の点で異なる。
- 2次元の画像であるテンソルを1次元のテンソルに変換する。
- 損失を求めるために正解データをone-hotベクトル化する。

MLPはその構造上、**2次元以上である画像データをその形状のまま入力として受け取る事ができない。**  
よって、画像を行や列で区切るなどして**1次元のテンソルに変換してからモデルに入力**する。  
今回は、形状変形のために`reshape()`を使おう。

また、  
モデルの出力は**各クラスの予測確率に相関したベクトル**であるのに対し、**正解ラベルはクラス名（0~9の値）**であるため、  
そのままではMSEを計算する事ができない。  
そこで、正解ラベルを「one-hotベクトルに変換」することで、モデルの予測結果との損失計算を可能にする。  
【one-hotベクトル化の例】
- 正解ラベル「1」 → `[0,1,0,0,0,0,0,0,0,0]` （=「1」である確率が1の確率ベクトル）
- 正解ラベル「7」 → `[0,0,0,0,0,0,0,1,0,0]` （=「7」である確率が1の確率ベクトル）

以上のことを踏まえて、  
例題を見ながら`train()`関数の実装方法を確認しよう。

【例題】 `train()`関数を実装する。  
まずは、`train()`関数の全体像を思い出そう。  
`train()`関数は、引数に`model`（学習するモデル）と`train_loader`（学習データローダ）を受け取り、  
`train_loader`を「バッチ=ある程度の数のデータのまとまり」ごとにループさせて学習を行う。  
この1ループは、次の流れで進む。
1. バッチごとのデータをモデルへ順伝播させる
2. 正解ラベルと比較して損失を計算する
3. 損失から最適化を行う
4. 次のバッチへ

今回は、これに加えて**「画像データの1次元化」**と**「正解ラベルのone-hotベクトル化」**を忘れないようにしよう。  
one-hotベクトル化には`torch.eye()`が使える。  
`torch.eye(クラス数)[バッチごとのデータ]`と記述することで、`バッチごとのデータ`の1要素に対してone-hotベクトル化を適用する事ができる。

In [ ]:
def train(model, train_loader):
    # 今は学習時であることを明示するコード
    model.train()
    for batch_imgs, batch_labels in train_loader:
        ## 画像データを1次元に変換
        batch_imgs = batch_imgs.reshape(-1, 28*28*1)
        ## 正解ラベルをone-hotベクトルへ変換
        labels = torch.eye(10)[batch_labels]

        ## 順伝播
        outputs = model(batch_imgs)
        ## 勾配を初期化（前回のループ時の勾配を削除）
        model.optimizer.zero_grad()
        ## 損失を計算
        loss = model.criterion(outputs, labels)
        ## 逆伝播で勾配を計算
        loss.backward()
        ## 最適化
        model.optimizer.step()
    return

以上のコードでは、学習自体はできるが学習過程を確認する事ができないので、  
「正答率」と「損失の合計」を出力できるように、以下のように改良する。  
追記部分は、ほとんどが「実用画像データ処理」を流用したものなので、コードを読めば理解できるだろう。

In [ ]:
def train(model, train_loader):
    # 今は学習時であることを明示するコード
    model.train()

    ### 追記部分1 ###
    # 正しい予測数、損失の合計、全体のデータ数を数えるカウンターの0初期化
    total_correct = 0
    total_loss = 0
    total_data_len = 0
    ### ###

    for batch_imgs, batch_labels in train_loader:
        ## 2次元データを1次元に変換
        batch_imgs = batch_imgs.reshape(-1, 28*28*1)
        ## 正解ラベルをone-hotベクトルへ変換
        labels = torch.eye(10)[batch_labels]

        ## 順伝播
        outputs = model(batch_imgs)
        ## 勾配を初期化（前回のループ時の勾配を削除）
        model.optimizer.zero_grad()
        ## 損失を計算
        loss = model.criterion(outputs, labels)
        ## 逆伝播で勾配を計算
        loss.backward()
        ## 最適化
        model.optimizer.step()
        
        ### 追記部分2 ###
        # 正答率を求める
        _, pred_labels = torch.max(outputs, 1)
        batch_size = len(batch_labels)
        for i in range(batch_size):
            total_data_len += 1
            if pred_labels[i] == batch_labels[i]:
                total_correct += 1
        total_loss += loss.item()
    accuracy = total_correct/total_data_len*100
    loss = total_loss/total_data_len
    return accuracy, loss
    ### ###

In [ ]:
# モデルを宣言する
model = mlp_net()

# 学習させ、その結果を表示する
acc, loss = train(model, train_loader)
print(f'正答率: {acc}, 損失: {loss}')

- ```
正答率： {95.0前後}, 損失: {0.0002前後}
```
と表示されていれば成功だ。

【問題】 新しいMLPモデル`mlp_net_2()`をクラスとして定義して、学習、結果を例題と同様に表示しよう。  
ただし、以下の条件にあるようにハイパーパラメータを指定すること。
- 宣言する`mlp_net_2()`のインスタンス名（変数名）は`model_2`とすること
- 中間層のノードの数を`256`にする
- `optim.Adam()`の引数`lr`に`0.01`を指定する

In [ ]:
class mlp_net_2(nn.Module):
  def __init__(self):
    super().__init__()

    self.fc1 = nn.Linear(784, 256)
    self.fc2 = nn.Linear(256, 10)

    self.criterion = nn.MSELoss()
    self.optimizer = optim.Adam(self.parameters(), lr=0.01)

  def forward(self, x):
    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    return x

In [ ]:
# モデルを宣言する
model_2 = mlp_net_2()

# 学習させ、その結果を表示する
acc, loss = train(model_2, train_loader)
print(f'正答率: {acc}, 損失: {loss}')

- ```
正答率： {90.0前後}, 損失: {0.00025前後}
```
と表示されていれば成功だ。
- 恐らく、例題よりも正答率は下がり、損失は大きくなったのではないだろうか。  
このように、ハイパーパラメータは学習に大きく影響するということを覚えておこう。

## test()関数の実装
次は、テストデータを使った評価を行う`test()`関数を実装する。  `train()`関数から「損失計算」や「最適化」の要素を取り除こう。  

In [ ]:
# test関数の実装

## モデルの予測結果を出力する
## ここでいう予測結果とは、確率ベクトルではなく、予測された「値」だ。
## バッチごとに出力するので実際はバッチサイズだけ要素をもつテンソルになる
def predict(model, batch_imgs):
    outputs = model(batch_imgs.reshape(-1, 28*28*1))
    _, pred_labels = torch.max(outputs, axis=1)
    return pred_labels

# test()の実装
def test(model, data_loader):
    # モデルを評価モードにする
    model.eval()
    # 正しい予測数、全体のデータ数を数えるカウンターの0初期化
    total_data_len = 0
    total_correct = 0
    for batch_imgs, batch_labels in data_loader:
        # 順伝播（=予測）
        pred_labels = predict(model, batch_imgs)
        # 集計
        batch_size = len(pred_labels)
        for i in range(batch_size):
            total_data_len += 1
            if pred_labels[i] == batch_labels[i]:
                total_correct += 1
    acc = 100.0 * total_correct/total_data_len
    return acc

【例題】 学習させた`mlp_net`と`test_loader`を使って、テストを行う。

In [ ]:
test_acc = test(model, test_loader)
print(test_acc)

- 正答率がおよそ`95%`前後になれば成功だ。